# Word Embeddings
Word Embeddings es una técnica que consiste en asignarle a cada palabra un vector de características que representa el concepto asociado a la palabra. En el siguiente ejemplo, utilizaremos el modelo preentrenado utilizando la técnica Word2Vec[1] utilizando artículos de [Google News](https://code.google.com/archive/p/word2vec/). 

[1] Tomas Mikolov, Kai Chen, Greg Corrado, and Jeffrey Dean. Efficient Estimation of Word Representations in Vector Space. In Proceedings of Workshop at ICLR, 2013.


In [ ]:
%matplotlib inline
import pickle
import numpy as np
from matplotlib import pyplot as plt
import gensim
import gdown

In [ ]:
gdown.download("https://drive.google.com/uc?id=0B7XkCwpI5KDYNlNUTTlSS21pQmM", "GoogleNews-vectors-negative300.bin.gz", quiet=False)
!gunzip -d GoogleNews-vectors-negative300.bin.gz

In [ ]:
model = gensim.models.KeyedVectors.load_word2vec_format('GoogleNews-vectors-negative300.bin', binary=True)

In [ ]:
words = ['king', 'queen', 'man', 'woman']
print('Vector king: {}'.format(model.wv['king']))

vec = np.empty((4, 300))
for i, w in enumerate(words):
    vec[i, :] = model.wv[w]

from sklearn.manifold import TSNE

x = TSNE().fit_transform(vec)
fig, ax = plt.subplots()
ax.scatter(x[:, 0], x[:, 1])
for i, w in enumerate(words):
    ax.annotate(w, (x[i, 0], x[i, 1]))

plt.show()

In [ ]:
print(model.most_similar(positive=['woman', 'king'], negative=['man']))

In [ ]:
del model

## Entrenando word embeddings
Una de las técnicas utilizadas para entrenar estos word embeddings se suele definir un clasificador que intenta calcular la probabilidad de la palabra w dado un contexto C, es decir, un conjunto de palabras cercanas. Es decir, nuestro clasificador intenta:

$$h(w,C) \approx P(w|C)$$

En general esto se hace a través de una red neuronal superficial.

<img src="https://i.stack.imgur.com/OpupG.png" alt="Shallow NN" style="width: 400px;"/>

> Fig. 1: Red Neuronal superficial <br>

En esta arquitecturas, cada palabra se representa mediante un ID único, donde cada ID a su vez puede ser traducido en un vector de todos ceros, menos un uno en la posición de ID. Supongamos un vocabulario restringido $VOC=\{P1, P2, P3, P4\}$, entonces a la parabra P2 se le asigna el ID 2 y el vector $V(P2)=(0,1,0,0)$. Esta forma de codificación, se conoce como hot-one. El embeddings de las palabras es la matríz de pesos $W$ de la primera capa densa, donde cada columna representa a la palabra asociada. Es decir, el embedding de la palabra P2 sería $W[2,:]$. Es importante notar que:

$$V(P2) \cdot W = (0, 1, 0,0) \cdot \left[\begin{array}{c}
W[1,1]\ldots W[1,n]\\
W[2,1]\ldots W[2,n]\\
W[3,1]\ldots W[3,n]\\
W[4,1]\ldots W[4,n]
\end{array}\right] = w[2,:]$$

## Negative Sampling
Para el entrenamiento es sencillo conseguir ejemplos positivos a partir del texto. Por ejemplo si consideramos el texto "La Argentina está organizada como un Estado federal descentralizado, integrado desde 1994 por un Estado nacional y 24 estados autogobernados", obtenido del artículo sobre [Argentinina en Wikipedia](https://es.wikipedia.org/wiki/Argentina), es facil ver que en un contexto de 4 palabras alrededor de "estado" están las palabras "está", "organizado", "como", "un", "federal", "descentralizado", "integrado" y "desde". Sin embargo, no hay ejemplos de palabras no asociadas con estado, por lo que se suele utilizar la técnica de sampleo negativos, es decir seleccionar palabras aleatores para generar nuestra muestra negativa.
Por ejemplo, consideremos:
* Un vocabulario con 10 palabras.
* Una oración compuesta por las palabras [0, 1, 2, 3, 4, 5, 6].
* Un contexto de 2 palabras. Por ejemplo, el contexto de la palabra 3 serían 1, 2, 4, 5.
* Un rate de positive samples y negative samples de 1.
Podríamos generar nuestra muestra de entrenamiento de la siguiente manera:

In [ ]:
from tensorflow.keras.preprocessing.sequence import skipgrams
sentence = list(range(0, 7))
print(sentence)
x, y = skipgrams(sentence, 10, window_size=2, negative_samples=1.0, shuffle=False)
print('Skipgrams: ')
print(x)
print(y)

Puede observarse que en algunos casos una instancia aparece como positiva y negativa al mismo tiempo, sin embargo este problema se mitiga a medida de que el vocabulario se hace más grande. 
Obviamente, para poder entrenar se necesitan más datos, para esto utilizaremos un conjunto de datos de noticias. Es importante notar que, si bien están clasificadas, esto no es de importancia debido a que Word2Vec es un algoritmo de aprendizaje no supervisado. Esto significa que no necesita datos etiquetados para aprender.
El siguiente código levanta los datos y los formatea correctamente. Solo considera palabras con 5 o más repeticiones y le aplica stemming para reducir el espacio de entrenamiento. Los skipgrams consirean una ventana de 5 elemento y una proporción de 5 ejemplos negativos por cada ejemplo verdadero.

In [ ]:
!pip install bs4
!pip install tqdm
from tqdm.notebook import tqdm
from bs4 import BeautifulSoup
import re 

def preprocessor(text):
    # remove HTML tags
    text = BeautifulSoup(text, 'html.parser').get_text()
    
    # regex for matching emoticons, keep emoticons, ex: :), :-P, :-D
    r = '(?::|;|=|X)(?:-)?(?:\)|\(|D|P)'
    emoticons = re.findall(r, text)
    text = re.sub(r, '', text)
    
    # convert to lowercase and append all emoticons behind (with space in between)
    # replace('-','') removes nose of emoticons
    text = re.sub('[\W]+', ' ', text.lower()) + ' ' + ' '.join(emoticons).replace('-','')
    return text


import nltk
from nltk.corpus import stopwords
from nltk.stem.porter import PorterStemmer

#Baja los stopwords
nltk.download('stopwords')
stop = stopwords.words('english')

def tokenizer_stem_nostop(text):
    porter = PorterStemmer()
    return [porter.stem(w) for w in re.split('\s+', text.strip()) \
            if w not in stop and re.match('[a-zA-Z]+', w)]


from sklearn.datasets import fetch_20newsgroups

categories = [
    'rec.autos',
    'rec.motorcycles',
    'rec.sport.baseball',
    'rec.sport.hockey',
    'sci.crypt',
    'sci.electronics',
    'sci.med',
    'sci.space',
]

remove = ('headers', 'footers', 'quotes')

newsgroups = fetch_20newsgroups(subset='all', categories=categories,
                                     shuffle=True, random_state=0,
                                     remove=remove)

from collections import Counter, deque

def process_corpus(data, words_id=None, min_reps=5):
    corpus = []
    for text in tqdm(data):
        corpus.append(tokenizer_stem_nostop(preprocessor(text)))
        
    if words_id is None:
        #Cuenta palabras en el corpus
        words = Counter()

        for s in corpus:
            for w in s:
                words[w] += 1

        #Elimina palabras con menos de 5 repeticiones
        words_id = {}
        id_next = 0
        for w, c in words.items():
            if c >= min_reps:
                words_id[w] = id_next
                id_next += 1

    id_words = { v:k for k, v in words_id.items()}
    corpus_id = [[words_id[w] for w in s if w in words_id] for s in corpus]
    return corpus_id, words_id, id_words

In [ ]:
x = deque()
y = deque()

corpus_id, words_id, id_words = process_corpus(newsgroups.data)

#Crea los skipgrams de entrenamiento
from tensorflow.keras.preprocessing.sequence import skipgrams
for s in tqdm(corpus_id):
    x1, y1 = skipgrams(s, len(id_words), window_size=3, negative_samples=5)
    x.extend(x1)
    y.extend(y1)


x1, x2 = zip(*x)
import numpy as np
x1 = np.asarray(x1)
x2 = np.asarray(x2)
y = np.asarray(y, dtype=np.float32)

print('Vocabulario: {}'.format(len(id_words)))
print('Skipgrams: {}'.format(x1.shape[0]))

## Implementando el clasificador
Si bien se desea implementar una red densa, esto es ineficiente. Hay que notar que la representación one-hot hace que la multiplicación sea efectivamente acceder una fila de la matriz $W$, en general:

$$V(Pi) \cdot W = (0, \ldots, 1, \ldots,0) \cdot \left[\begin{array}{c}
W[1,1]\ldots W[1,n]\\
W[2,1]\ldots W[2,n]\\
\ddots \ddots \ddots\\
W[m,1]\ldots W[m,n]
\end{array}\right] = w[i,:]$$

La capa Embeddings de Keras funciona de esta manera, es decir, recibe vector de ids de palabras(enteros) y retorna una matriz donde cada fila representa el embedding de la palabra. Considerando esto, la matriz de embeddings se puede entrenar de la siguiente manera:

In [ ]:
from tensorflow.keras.layers import dot, Embedding, Input, Activation, Flatten
from tensorflow.keras.models import Model
from tensorflow.keras.callbacks import ReduceLROnPlateau, EarlyStopping
from tensorflow.keras.optimizers import SGD
from tensorflow.keras import backend as K

#Función de error basada en log-likelihood
def minus_max_likelihood(y_true, y_pred):
    max_like = y_true * K.log(1+ K.exp(-y_pred)) + (1 - y_true) * K.log(1+ K.exp(y_pred)) 
    return max_like

context_emb = Embedding(len(id_words), 64, name='Emb_context')
target_emb = Embedding(len(id_words), 64, name='Emb_target')

context = Input((1,), name='context')
emb = context_emb(context)
target = Input((1,), name='target')
embT = target_emb(target)
lam = dot([emb, embT], axes=(-1))
lam = Flatten()(lam) 
#lam = Activation('sigmoid')(lam)

model = Model(inputs=[context, target], outputs=lam)
model.compile('adam', minus_max_likelihood)
#model.compile('adam', 'binary_crossentropy')
model.summary()

#Entrenamos poco 
model.fit([x1, x2], y.astype(np.float32), epochs=1, batch_size=1000)
#Obtención de los embeddings
vectors = K.get_value(target_emb.embeddings)
#Recuperar memoria
del context_emb
del target_emb

In [ ]:
print(vectors.shape)
print(words_id['car'])
print(vectors[words_id['car'], :])
print(vectors[words_id['car'], :].shape)

np.dot(vectors[words_id['car'], :], vectors[words_id['ford'], :]) 

## Viendo algún resultado
Como antes podemos asumir que las palabras similares están cercanas en el espacio de vectores. Esta similitud la podemos medir por similitud del coseno. Podemos ver palabras similares a "car", "ford", "law":

In [ ]:
def cos(v1, v2):
    return np.dot(v1, v2.T) / (np.dot(v1, v1.T) ** 0.5 * np.sum(v2 * v2, axis=-1) ** 0.5)


def nearest(voc, wv, top=11):
    dist = cos(wv, voc)
    a = range(len(dist))
    a = sorted(a, key=lambda x: dist[x], reverse=True)
    return a[0:top], [dist[x] for x in a[0:top]]

print('Similares a car:')
for i, d in zip(*nearest(vectors, vectors[words_id['car'], :])):
    print('\t{} {}'.format(id_words[i], d))

print('Similares a ford:')
for i, d in zip(*nearest(vectors, vectors[words_id['ford'], :])):
    print('\t{} {}'.format(id_words[i], d))

print('Similares a law:')
for i, d in zip(*nearest(vectors, vectors[words_id['law'], :])):
    print('\t{} {}'.format(id_words[i], d))
    

In [ ]:
del x
del y
del x1
del x2
del vectors

## GloVe
[GloVe](https://nlp.stanford.edu/projects/glove/) es otro método para entrenar embeddings basado en matriz de coocurrencias. El objetivo de la red en este caso es predecir la cantidad de coocurrencias de dos palabras.

In [ ]:
from collections import defaultdict


def bigram_count(token_list, window_size, cache):
    sentence_size = len(token_list)

    for central_index, central_word_id in enumerate(token_list):
        for distance in range(1, window_size + 1):
            if central_index + distance < sentence_size:
                first_id, second_id = sorted([central_word_id, token_list[central_index + distance]])
                cache[first_id][second_id] += 1.0 / distance
    pass


def build_cooccurrences(sequences, cache, window=3):
    for seq in tqdm(sequences):
        bigram_count(token_list=seq, cache=cache, window_size=window)


def process_coocurrence_matrix(sentences, window_size=3):
    cache = defaultdict(lambda : defaultdict(int))

    build_cooccurrences(sentences, cache=cache, window=window_size)
    first, second, x_ijs = deque(), deque(), deque()

    for first_id in cache.keys():
        for second_id in cache[first_id].keys():
            x_ij = cache[first_id][second_id]

            first.append(first_id)
            second.append(second_id)
            x_ijs.append(x_ij)

            first.append(second_id)
            second.append(first_id)
            x_ijs.append(x_ij)

    return np.array(first), np.array(second), np.array(x_ijs)

In [ ]:
x1, x2, y = process_coocurrence_matrix(corpus_id)

In [ ]:
from tensorflow.keras.layers import Input, Embedding, Dot, Reshape, Add
from tensorflow.keras.models import Model
import tensorflow.keras.backend as K

def custom_loss(y_true, y_pred, a = 3.0/4.0, X_MAX=100):
    """
    This is GloVe's loss function
    :param y_true: The actual values, in our case the 'observed' X_ij co-occurrence values
    :param y_pred: The predicted (log-)co-occurrences from the model
    :return: The loss associated with this batch
    """
    return K.sum(K.pow(K.clip(y_true / X_MAX, 0.0, 1.0), a) * K.square(y_pred - K.log(y_true)), axis=-1)


def glove_model(vocab_size=10, vector_dim=64):
    """
    A Keras implementation of the GloVe architecture
    :param vocab_size: The number of distinct words
    :param vector_dim: The vector dimension of each word
    :return:
    """
    input_target = Input((1,), name='central_word_id')
    input_context = Input((1,), name='context_word_id')

    central_embedding = Embedding(vocab_size+1, vector_dim, input_length=1, name='central_emb')
    central_bias = Embedding(vocab_size+1, 1, input_length=1, name='central_bias')

    context_embedding = Embedding(vocab_size, vector_dim, input_length=1, name='context_emb')
    context_bias = Embedding(vocab_size, 1, input_length=1, name='context_bias')

    vector_target = central_embedding(input_target)
    vector_context = context_embedding(input_context)

    bias_target = central_bias(input_target)
    bias_context = context_bias(input_context)

    dot_product = Dot(axes=-1)([vector_target, vector_context])
    dot_product = Reshape((1, ))(dot_product)
    bias_target = Reshape((1,))(bias_target)
    bias_context = Reshape((1,))(bias_context)

    prediction = Add()([dot_product, bias_target, bias_context])

    model = Model(inputs=[input_target, input_context], outputs=prediction)
    model.compile(loss=custom_loss, optimizer='adam')

    return model

In [ ]:
model = glove_model(len(words_id), 64)
model.summary()

model.fit([x1, x2], y, epochs=5, batch_size=512)

In [ ]:
from keras.utils import plot_model

plot_model(model, show_shapes=True, show_layer_names=True, to_file='model.png')
from IPython.display import Image
Image(retina=True, filename='model.png')

In [ ]:
#Obtención de los embeddings
vectors = K.get_value(model.layers[2].embeddings)
#Recuperar memoria
#del model

print('Similares a car:')
for i, d in zip(*nearest(vectors, vectors[words_id['car'], :])):
    print('\t{} {}'.format(id_words[i], d))

print('Similares a ford:')
for i, d in zip(*nearest(vectors, vectors[words_id['ford'], :])):
    print('\t{} {}'.format(id_words[i], d))

print('Similares a law:')
for i, d in zip(*nearest(vectors, vectors[words_id['law'], :])):
    print('\t{} {}'.format(id_words[i], d))

In [ ]:
del x1
del x2
del y

# Redes Neuronales Recurrentes
Tradicionalmente, una de las formas de representar texto para machine learning es utilizar "bag-of-words", o alguna variación que cuente la frecuencia de las palabras en el texto y corpus, como [TF-IDF](https://en.wikipedia.org/wiki/Tf%E2%80%93idf), ver documentación de [sk-learn](http://scikit-learn.org/stable/modules/feature_extraction.html#text-feature-extraction). Supongamos que queremos entrenar un clasificador de opiniones y tenemos la frase "Pedro no es tonto, es inteligente" categorizada como positiva. La frase podría representarse en un vector donde cada posición representa una palabra del vocabulario y su valor es la fercuencia en la frase. Entonces, nuestra frase sería representada por un vector ralo: {'Pedro': 1, 'no':1, 'es': 2, 'tonto': 1, 'inteligente': 1}. Pero la frase "Pedro no es inteligente, es tonto" tendría la misma representación pero sentido completamente opuesto. Este es un ejemplo donde casos donde las características de las instancias tiene relación de orden, es decir, no basta con conocer las características de la instancia para representarla correctamente sino el orden.

Exiten redes neuronales que consideran esta información y se conocen como redes neuronales recurrentes. En estas redes, el valor de salida depende de los valores de entrada procesados de forma secuencial.

<img src="https://upload.wikimedia.org/wikipedia/commons/b/b5/Recurrent_neural_network_unfold.svg" alt="Shallow NN" style="width: 400px;"/>

> Fig. 2: Red Neuronal Recurrente. Imagen: [Wikipedia](https://en.wikipedia.org/wiki/Recurrent_neural_network) <br>

Como se ve, en estas redes, una instancia es una sequencia de elementos. En el caso de texto, la secuencia puede ser una secuencia de vectores embedding generados con alguna técnica como Word2Vec, o entrenados directamente en la red.

Una de las capas recurrentes más comuenmente usada es la LSTM. 

<img src="https://cdn-images-1.medium.com/max/2000/0*LyfY3Mow9eCYlj7o." alt="Shallow NN" style="width: 400px;"/>

> Fig. 2: LSTM. Imagen: [Codeburst](https://codeburst.io/generating-text-using-an-lstm-network-no-libraries-2dff88a3968) <br>

In [ ]:
#Cargar datos en formato texto
import pickle
import gzip

#Cargamos bibliotecas y demás
!pip install tqdm
%matplotlib inline
import matplotlib.pyplot as plt
import numpy as np
from tensorflow.keras.datasets import mnist
from sklearn.metrics import accuracy_score
from tensorflow.keras.layers import Input, Dense, Conv2D, Flatten
from tensorflow.keras.models import Model 
from tensorflow.keras.utils import to_categorical
from tensorflow.keras import backend as K

from tqdm.notebook import tqdm
import os.path
while not os.path.exists('imdb.pkl.gzip'):
    #Si no está el archivo hay que subirlo. Solo para Google Colab!!
    from google.colab import files
    uploaded = files.upload()
    for fn in uploaded.keys():
        print('User uploaded file "{name}" with length {length} bytes'.format(name=fn, length=len(uploaded[fn])))

(x_train, y_train), (x_test, y_test) = pickle.load(gzip.open('imdb.pkl.gzip', 'rb'))

In [ ]:
print(x_train[0:3])
print(y_train[0:3])

In [ ]:
#Reformateamos usando la estrategia definida arriba.
x_train, words_id, _ = process_corpus(x_train)
x_test, _, _ = process_corpus(x_test, words_id)
idx = words_id

In [ ]:
print(x_train[0:3])
print(y_train[0:3])

In [ ]:
print(idx)

In [ ]:
#Creamos una red neuronal recurrente con embedddings
from tensorflow.keras.layers import Embedding, Dense, LSTM, Input, Bidirectional
from tensorflow.keras.models import Model

i = Input((None,))
d = Embedding(len(idx) + 1, 150, mask_zero=True)(i)
d = LSTM(150, return_sequences=True, dropout=0.2, recurrent_dropout=0.2)(d)
d = LSTM(150, return_sequences=False)(d)
d = Dense(100, activation='relu')(d)
d = Dense(1, activation='sigmoid')(d)

model = Model(inputs=i, outputs=d)
model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['binary_accuracy'])
model.summary()

In [ ]:
h = model.fit(x_train, y_train, epochs=2, batch_size=256, validation_data=(x_test, y_test))

In [ ]:
#Testeamos la red neuronal
from tensorflow.keras.preprocessing.sequence import pad_sequences
x_train = pad_sequences(x_train, 50)
x_test = pad_sequences(x_test, 50)

h = model.fit(x_train, y_train, epochs=5, batch_size=256, validation_data=(x_test, y_test))

In [ ]:
print(x_train.shape)
print(x_train[10,:])

In [ ]:
%matplotlib inline
import pickle
import numpy as np
from matplotlib import pyplot as plt

print('Loss')
plt.plot(h.history['loss'], 'r-')
plt.plot(h.history['val_loss'], 'b-')
plt.show()

print('Accuracy')
plt.plot(h.history['binary_accuracy'], 'r-')
plt.plot(h.history['val_binary_accuracy'], 'b-')
plt.show()

## Auto-Nietzsche
En esta sección utilizaremos una red neuronal recurrente para autogenerar texto. Para esto, utilizaremos el ejemplo propuesto por [Keras](https://github.com/keras-team/keras/blob/master/examples/lstm_text_generation.py)

In [ ]:
from tensorflow.keras.callbacks import LambdaCallback
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense
from tensorflow.keras.layers import LSTM
from tensorflow.keras.optimizers import RMSprop
from tensorflow.keras.utils.data_utils import get_file
import numpy as np
import random
import sys
import io

path = get_file(
    'nietzsche.txt',
    origin='https://s3.amazonaws.com/text-datasets/nietzsche.txt')
with io.open(path, encoding='utf-8') as f:
    text = f.read().lower()
print('corpus length:', len(text))

chars = sorted(list(set(text)))
print('total chars:', len(chars))
char_indices = dict((c, i) for i, c in enumerate(chars))
indices_char = dict((i, c) for i, c in enumerate(chars))

# cut the text in semi-redundant sequences of maxlen characters
maxlen = 40
step = 3
sentences = []
next_chars = []
for i in range(0, len(text) - maxlen, step):
    sentences.append(text[i: i + maxlen])
    next_chars.append(text[i + maxlen])
print('nb sequences:', len(sentences))

print('Vectorization...')
x = np.zeros((len(sentences), maxlen, len(chars)), dtype=np.bool)
y = np.zeros((len(sentences), len(chars)), dtype=np.bool)
for i, sentence in enumerate(sentences):
    for t, char in enumerate(sentence):
        x[i, t, char_indices[char]] = 1
    y[i, char_indices[next_chars[i]]] = 1

print(sentences[0])
print(next_chars[0])
print(x[0,:,:])
print(y[0,:])
# build the model: a single LSTM
print('Build model...')
model = Sequential()
model.add(LSTM(128, input_shape=(maxlen, len(chars))))
model.add(Dense(len(chars), activation='softmax'))

optimizer = RMSprop(lr=0.01)
model.compile(loss='categorical_crossentropy', optimizer=optimizer, metrics=['accuracy'])


def sample(preds, temperature=1.0):
    # helper function to sample an index from a probability array
    preds = np.asarray(preds).astype('float64')
    preds = np.log(preds) / temperature
    exp_preds = np.exp(preds)
    preds = exp_preds / np.sum(exp_preds)
    probas = np.random.multinomial(1, preds, 1)
    return np.argmax(probas)


def on_epoch_end(epoch, _):
    # Function invoked at end of each epoch. Prints generated text.
    print()
    print('----- Generating text after Epoch: %d' % epoch)

    start_index = random.randint(0, len(text) - maxlen - 1)
    for diversity in [0.2, 0.5, 1.0, 1.2]:
        print('----- diversity:', diversity)

        generated = ''
        sentence = text[start_index: start_index + maxlen]
        generated += sentence
        print('----- Generating with seed: "' + sentence + '"')
        sys.stdout.write(generated)

        for i in range(400):
            x_pred = np.zeros((1, maxlen, len(chars)))
            for t, char in enumerate(sentence):
                x_pred[0, t, char_indices[char]] = 1.

            preds = model.predict(x_pred, verbose=0)[0]
            next_index = sample(preds, diversity)
            next_char = indices_char[next_index]

            generated += next_char
            sentence = sentence[1:] + next_char

            sys.stdout.write(next_char)
            sys.stdout.flush()
        print()

print_callback = LambdaCallback(on_epoch_end=on_epoch_end)
on_epoch_end(-1,None)

model.fit(x, y,
          batch_size=128,
          epochs=10,
          callbacks=[print_callback])

## Ejercicio
Implemente una red neuronal para resolver [Facebook bAbI task](https://research.fb.com/downloads/babi/), para el caso una frase, una pregunta; y dos frases, una pregunta.
Pasos a seguir:

1. Formatear los datos para la entrada de la red neuronal.
1. Definir la arquitectura.
1. Entrenar y testear.

Se recomienda usar una entrada diferente para cada componente de la entrada. Por ejemplo, para el caso una frase, una pregunta:

```
x_sentence = ...#Oraciones
x_question = ...#Pregunta asociada a la oracion
y = ... #Respuesta

i_sentence = Input(...)
d_sentence = Capa_X()(i_sentence)
...
d_sentence = Capa_X()(d_sentence)

i_question = Input(...)
d_question = Capa_X()(i_question)
...
d_question = Capa_X()(d_question)

d = concatenate(d_sentence, d_question) #Hay otras opciones como add, multiply, dot...
d = ...
d = CapaFinal()(d)

model = Model(inputs=[i_sentence, i_question], outputs=d)
...

model.fit([x_sentence, x_question], y, ...)
```


# Attention
Este tipo de capas es la base de los transformers que son actualmente utilizados en arquitecturas como BERT o GTP-3.

In [ ]:
!wget https://ai.stanford.edu/~amaas/data/sentiment/aclImdb_v1.tar.gz
!tar -xf aclImdb_v1.tar.gz

In [ ]:
import os
from sklearn.utils import shuffle
import numpy as np

def load_imdb_ds(path, shuffle_ds=True, random_state=None):
    def load_text(path_sel):
        xa = []
        for f in os.listdir(path_sel):
            f = open(path_sel + os.sep + f, 'r')
            xa.append(next(f))
        return xa
    x = load_text(path + os.sep + 'pos')
    y = [1] * len(x)
    xn = load_text(path + os.sep + 'neg')
    x.extend(xn)
    y.extend([0] * len(xn))
    if shuffle_ds:
        shuffle(x, y, random_state=random_state)
    return x, y

x_train_text, y_train = load_imdb_ds('aclImdb/train', random_state=42)
x_test_text, y_test = load_imdb_ds('aclImdb/test', random_state=42)

In [ ]:
!pip install bs4
!pip install tqdm

from collections import Counter, deque
from tqdm.notebook import tqdm
from bs4 import BeautifulSoup
import re 

def preprocessor(text):
    # remove HTML tags
    text = BeautifulSoup(text, 'html.parser').get_text()
    
    # regex for matching emoticons, keep emoticons, ex: :), :-P, :-D
    r = '(?::|;|=|X)(?:-)?(?:\)|\(|D|P)'
    emoticons = re.findall(r, text)
    text = re.sub(r, '', text)
    
    # convert to lowercase and append all emoticons behind (with space in between)
    # replace('-','') removes nose of emoticons
    text = re.sub('[\W]+', ' ', text.lower()) + ' ' + ' '.join(emoticons).replace('-','')
    return text


import nltk
from nltk.corpus import stopwords
from nltk.stem.porter import PorterStemmer

#Baja los stopwords
nltk.download('stopwords')
stop = stopwords.words('english')

def tokenizer_stem_nostop(text):
    porter = PorterStemmer()
    return [porter.stem(w) for w in re.split('\s+', text.strip()) \
            if w not in stop and re.match('[a-zA-Z]+', w)]


def tokenizer_simple(text):
    return [w for w in re.split('\s+', text.strip()) \
            if re.match('[a-zA-Z]+', w)]


def process_corpus(data, words_id=None, min_reps=5, tokenizer=tokenizer_simple):
    corpus = []
    for text in tqdm(data):
        corpus.append(tokenizer(preprocessor(text)))
        
    if words_id is None:
        #Cuenta palabras en el corpus
        words = Counter()

        for s in corpus:
            for w in s:
                words[w] += 1

        #Elimina palabras con menos de 5 repeticiones
        words_id = {}
        id_next = 1
        for w, c in words.items():
            if c >= min_reps:
                words_id[w] = id_next
                id_next += 1

    id_words = { v:k for k, v in words_id.items()}
    corpus_id = [[words_id[w] for w in s if w in words_id] for s in corpus]
    return corpus_id, words_id, id_words

In [ ]:
x_train, words_id, id_words = process_corpus(x_train_text)
x_test, _, _ = process_corpus(x_test_text, words_id=words_id)
y_train = np.expand_dims(np.asarray(y_train), axis=-1)
y_test = np.expand_dims(np.asarray(y_test), axis=-1)

In [ ]:
from tensorflow.keras.preprocessing.sequence import pad_sequences
MAXLEN = 50

x_train = pad_sequences(x_train, maxlen=MAXLEN)
x_test = pad_sequences(x_test, maxlen=MAXLEN)

In [ ]:
from tensorflow.keras.layers import Attention, Add, GlobalAveragePooling1D, Dropout, Lambda, Embedding, Input, Dense
import tensorflow as tf
from tensorflow.keras.models import Model

from tensorflow.keras.layers import Attention, Add, GlobalAveragePooling1D, Dropout
import tensorflow as tf


i = Input((None,))
e = Embedding(len(words_id) + 1, 60, mask_zero=True, name='base_emb')(i)

dq = Dense(60)(e)
dk = Dense(60)(e)

att = Attention()([dq, dk])
attd = Dropout(0.1)(att)

d = GlobalAveragePooling1D()(attd)
d = Dropout(0.1)(d)
d = Dense(100)(d)
d = Dropout(0.1)(d)
d = Dense(1, activation='sigmoid')(d)


model = Model(i, d)
model.summary()
model.compile(loss='binary_crossentropy', optimizer='rmsprop', metrics=['binary_accuracy'])

In [ ]:
from keras.utils import plot_model

plot_model(model, show_shapes=True, show_layer_names=True, to_file='model.png')
from IPython.display import Image
Image(retina=True, filename='model.png')

In [ ]:
model.fit(x_train, y_train, epochs=3, validation_data=(x_test, y_test))

In [ ]:
from keras.utils import plot_model

plot_model(model, show_shapes=True, show_layer_names=True, to_file='model.png')
from IPython.display import Image
Image(retina=True, filename='model.png')

In [ ]:
x_exam = ['The movie was excelent. It is probably the best movie ever', 'The movie was not good']
x_exam_v, _, _ = process_corpus(x_exam, words_id)
x_exam_v = pad_sequences(x_exam_v, maxlen=MAXLEN)

In [ ]:
print(model.predict(x_exam_v))

In [ ]:
model_att = Model(i, [dq, dk])

vq, vk = model_att(x_exam_v)

In [ ]:
att_sal = tf.nn.softmax(tf.matmul(vq, vk, transpose_b=True))
print(att_sal[0,-10:, -10:])
print(att_sal[0,-5:, -5:])

In [ ]:
import matplotlib.pyplot as plt


plt.imshow(att_sal[0,...])
plt.show()
plt.imshow(att_sal[0, -10:, -10:])
plt.show()

In [ ]:
plt.imshow(att_sal[1,...])
plt.show()
plt.imshow(att_sal[1, -5:, -5:])
plt.show()

In [ ]:
print(x_test_text[0])
print(model.predict(np.expand_dims(x_test[0, :], axis=0)))
vq, vk = model_att(np.expand_dims(x_test[0, :], axis=0))
att_sal = tf.nn.softmax(tf.matmul(vq, vk, transpose_b=True))
plt.imshow(att_sal[0,...])
plt.show()